#Клонируем проект с github и переходим в указанную директорию

In [ ]:
!git clone https://github.com/jantic/DeOldify.git DeOldify 

fatal: destination path 'DeOldify' already exists and is not an empty directory.


In [ ]:
cd DeOldify

/content/DeOldify


#Установка необходимых зависимостей

In [ ]:
from deoldify import device
from deoldify.device_id import DeviceId
device.set(device=DeviceId.GPU0)

import torch

if not torch.cuda.is_available():
    print('GPU not available.')

GPU not available.


In [ ]:
!pip install -r colab_requirements.txt

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 214 kB 5.2 MB/s 
     |████████████████████████████████| 129 kB 42.8 MB/s 
     |████████████████████████████████| 1.9 MB 48.3 MB/s 
     |████████████████████████████████| 78 kB 5.8 MB/s 
  Created wheel for typing: filename=typing-3.7.4.3-py3-none-any.whl size=26325 sha256=55a816af66d3971c2ea30ef3f1061f8f544443018df0867bbf3de8aa1faae22a
  Stored in directory: /root/.cache/pip/wheels/35/f3/15/01aa6571f0a72ee6ae7b827c1491c37a1f72d686fd22b43b0e
Successfully built typing
  Attempting uninstall: fastai
    Found existing installation: fastai 1.0.61
    Uninstalling fastai-1.0.61:
      Successfully uninstalled fastai-1.0.61


In [ ]:
import fastai
from deoldify.visualize import *
import warnings
warnings.filterwarnings("ignore", category=UserWarning, message=".*?Your .*? set is empty.*?")

In [ ]:
!mkdir 'models'
!wget https://data.deepai.org/deoldify/ColorizeArtistic_gen.pth -O ./models/ColorizeArtistic_gen.pth

--2022-06-17 09:58:16--  https://data.deepai.org/deoldify/ColorizeArtistic_gen.pth
Resolving data.deepai.org (data.deepai.org)... 138.201.36.183
Connecting to data.deepai.org (data.deepai.org)|138.201.36.183|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 255144681 (243M) [application/octet-stream]
Saving to: ‘./models/ColorizeArtistic_gen.pth’

./models/ColorizeAr 100%[===================>] 243.32M  16.6MB/s    in 11s     

2022-06-17 09:58:28 (21.5 MB/s) - ‘./models/ColorizeArtistic_gen.pth’ saved [255144681/255144681]



In [ ]:
!wget https://media.githubusercontent.com/media/jantic/DeOldify/master/resource_images/watermark.png -O ./resource_images/watermark.png

--2022-06-17 09:58:31--  https://media.githubusercontent.com/media/jantic/DeOldify/master/resource_images/watermark.png
Resolving media.githubusercontent.com (media.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to media.githubusercontent.com (media.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 9210 (9.0K) [image/png]
Saving to: ‘./resource_images/watermark.png’

./resource_images/w 100%[===================>]   8.99K  --.-KB/s    in 0s      

2022-06-17 09:58:31 (68.5 MB/s) - ‘./resource_images/watermark.png’ saved [9210/9210]



In [ ]:
colorizer = get_image_colorizer(artistic=True)

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:490: UserWarning: This DataLoader will create 8 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))
Downloading: "https://download.pytorch.org/models/resnet34-b627a593.pth" to /root/.cache/torch/hub/checkpoints/resnet34-b627a593.pth


  0%|          | 0.00/83.3M [00:00<?, ?B/s]

#Инструкция по использованию 

### source_url or source_path
В файле visualize.py реализованы две функции plot_transformed_image_from_url и plot_transformed_image, где первой передается url в качестве доступа к изображению, а второй path к изображению. Для пакетной обработки фото будет использована именно вторая функция, так как исходные изображения хранятся на google drive и url можно получить лишь на хранилище этого изображения.

### render_factor
Коэффициент рендеринга был эмпирически определен как "12", но как выяснилось в дальнейшем, почти каждое третье изображение различно по контрастности с отсальными, поэтому в идеале необходимо разбить исходные изображения в группы по степени контрастности и подбирать значение рендеринга индивидуально.

### watermarked
False или True напротив вотермарки активируют или деактивируют ее. Здесь она представлена как авторская задумка, что ваши фото были обработаны с помощью решения DeOldify.


In [ ]:
from PIL import Image
import os

In [ ]:
render_factor = 12  
watermarked = False 
images_names = os.listdir('/your_images_path/') # тут свою директорию с картинками

for name in images_names:
    try:
        path = f'/your_images_path/{name}'
        image_path = colorizer.plot_transformed_image(path, render_factor=render_factor, watermarked=watermarked)
    except Exception:
        print('error')

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


#Сравнение коэффицентов рендеринга на выбранном изображении

In [ ]:
for i in range(10,40,2):
    colorizer.plot_transformed_image('test_images/image.png', render_factor=i, display_render_factor=True, figsize=(8,8))

#Перемещение изображение из директории result_images на google drive

In [ ]:
import os
import shutil

folder_from = r'/content/DeOldify/result_images'
folder_to = r'/your_images_path/'

for f in os.listdir(folder_from):
    if os.path.isfile(os.path.join(folder_from, f)):
        shutil.copy(os.path.join(folder_from, f), os.path.join(folder_to, f))
    if os.path.isdir(os.path.join(folder_from, f)):
        os.system(f'rd /S /Q {folder_to}\\{f}')
        shutil.copytree(os.path.join(folder_from, f), os.path.join(folder_to, f))

#При необходимости можно очистить директорию result_images

In [ ]:
import os, shutil
folder = '/content/DeOldify/result_images'
for filename in os.listdir(folder):
    file_path = os.path.join(folder, filename)
    try:
        if os.path.isfile(file_path) or os.path.islink(file_path):
            os.unlink(file_path)
        elif os.path.isdir(file_path):
            shutil.rmtree(file_path)
    except Exception as e:
        print('Failed to delete %s. Reason: %s' % (file_path, e))